```
pandas==2.2.2
torch==2.6.0+cu124
torchaudio==2.6.0+cu124
turchvision==0.21.0+cu124
pillow==11.2.1
transformers==4.53.2
numpy==2.0.2
tqdm==4.67.1
scikit-learn==1.6.1
peft==0.16.0
```
python==3.11.13

OS==Ubuntu 22.04.4 LTS

In [ ]:
!unzip -qq scpc_data.zip

In [ ]:
!unzip -qq visual7w_images.zip

In [ ]:
!unzip -qq dataset_v7w_telling.zip

In [5]:

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import numpy as np
from tqdm.notebook import tqdm
import os
import logging
from sklearn.model_selection import train_test_split
import random
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training
from torch.amp import autocast, GradScaler
import warnings
warnings.filterwarnings('ignore')


In [6]:
seed = 41

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
#torch.backends.cudnn.deterministic = True
#torch.backends.cudnn.benchmark = False

In [7]:
class Config:
    def __init__(self):
        self.model_name = "laion/CLIP-ViT-L-14-laion2B-s32B-b82K"
        self.processor_name = "laion/CLIP-ViT-L-14-laion2B-s32B-b82K"
        self.train_csv_path = 'train_combined.csv'
        self.test_csv_path = 'test.csv'
        self.sample_submission_path = 'sample_submission.csv'
        self.output_submission_path = 'submission_pluto.csv'
        self.image_base_path = './'
        self.batch_size = 64  # 배치 크기 증가
        self.num_epochs = 6
        self.learning_rate = 1e-4  # 학습률 감소
        self.weight_decay = 1.5e-2
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.num_choices = 4
        self.validation_split = 0.15
        self.patience = 3  # Early stopping
        self.save_best_model = True
        self.model_save_path = 'best_clip_vqa_model.pth'
        # LoRA 설정
        self.lora_r = 32  # rank
        self.lora_alpha = 64  # alpha
        self.lora_dropout = 0.1

        self.lora_target_modules = [
            "q_proj",
            "v_proj",
            #"k_proj",
            "out_proj",  # attention layers
           # "fc1", "fc2",  # feed forward layers
           # "visual_projection", #"text_projection"  # projection layers
        ]

In [8]:
config = Config()
print(f"Using device: {config.device}")
print(f"Model: {config.model_name}")
print(f"Batch size: {config.batch_size}")
print(f"Learning rate: {config.learning_rate}")

Using device: cuda
Model: laion/CLIP-ViT-L-14-laion2B-s32B-b82K
Batch size: 64
Learning rate: 0.0001


In [9]:
df = pd.read_csv(config.test_csv_path)
max_length = df['Question'].astype(str).str.len().max()
print("최대 글자 수(Q):", max_length)

max_length = df['A'].astype(str).str.len().max()
print("최대 글자 수(A):", max_length)

max_length = df['B'].astype(str).str.len().max()
print("최대 글자 수(B):", max_length)

max_length = df['C'].astype(str).str.len().max()
print("최대 글자 수(C):", max_length)

max_length = df['D'].astype(str).str.len().max()
print("최대 글자 수(D):", max_length)

최대 글자 수(Q): 117
최대 글자 수(A): 91
최대 글자 수(B): 102
최대 글자 수(C): 86
최대 글자 수(D): 93


In [10]:
max_word_count = df['Question'].astype(str).str.split().str.len().max()
print("최대 단어 수(Question):", max_word_count)

max_word_count = df['A'].astype(str).str.split().str.len().max()
print("최대 단어 수(A):", max_word_count)

max_word_count = df['B'].astype(str).str.split().str.len().max()
print("최대 단어 수(B):", max_word_count)

max_word_count = df['C'].astype(str).str.split().str.len().max()
print("최대 단어 수(C):", max_word_count)

max_word_count = df['D'].astype(str).str.split().str.len().max()
print("최대 단어 수(D):", max_word_count)

최대 단어 수(Question): 24
최대 단어 수(A): 16
최대 단어 수(B): 15
최대 단어 수(C): 16
최대 단어 수(D): 17


In [11]:
import json
def append_to_existing_csv_pandas(json_file_path, existing_csv_path, starting_id=None):
    """
    기존 CSV 파일에 새로운 데이터를 추가하는 함수 (pandas 사용)
    """

    # 기존 CSV 파일 읽기
    existing_df = pd.read_csv(existing_csv_path, encoding='utf-8')

    # starting_id가 지정되지 않으면 기존 데이터의 마지막 ID + 1로 설정
    if starting_id is None:
        starting_id =100

    # JSON 파일 읽기
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    rows = []
    current_id = starting_id

    # 각 이미지와 QA 쌍을 처리
    for image in data['images']:
        for qa in image['qa_pairs']:
            choices = qa['multiple_choices'].copy()
            answer = qa['answer']

            if answer in choices:
                choices.remove(answer)

            while len(choices) < 3:
                choices.append("")

            choices = choices[:3]
            random.shuffle(choices)
            choices.append(answer)

            row = {
                'ID': current_id,
                'img_path': f'./images/{image["filename"]}',
                'Question': qa['question'],
                'A': choices[0] if choices[0] else "",
                'B': choices[1] if choices[1] else "",
                'C': choices[2] if choices[2] else "",
                'D': choices[3],
                'answer': 'D'
            }

            rows.append(row)
            current_id += 1

    # 새로운 DataFrame 생성
    new_df = pd.DataFrame(rows)

    # 기존 데이터와 합치기
    combined_df = pd.concat([existing_df, new_df], ignore_index=True)

    # CSV 파일로 저장
    combined_df.to_csv('train_combined.csv', index=False, encoding='utf-8')

    print(f"추가 완료: {len(new_df)}개의 행이 기존 CSV에 추가되었습니다.")
    print(f"전체 데이터: {len(combined_df)}개 행")


In [12]:
append_to_existing_csv_pandas('dataset_v7w_telling.json','train.csv')

추가 완료: 139868개의 행이 기존 CSV에 추가되었습니다.
전체 데이터: 139928개 행


In [13]:
class VQADataset(Dataset):
    def __init__(self, df, processor, image_base_path, is_train=True):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.image_base_path = image_base_path
        self.is_train = is_train
        self.num_choices = config.num_choices

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_base_path, row["img_path"])
        question = row["Question"]
        choices = [row["A"], row["B"], row["C"], row["D"]]

        try:
            image = Image.open(image_path).convert("RGB")
        except Exception as e:
            #logger.error(f"Error loading image {image_path}: {e}")
            # 기본 이미지 생성 (검은색 이미지)
            print('error: Unable to open image')
            image = Image.new('RGB', (224, 224), color='black')

        # 질문과 각 선택지를 결합한 텍스트 생성
        texts = [f"Question: {question} Answer: {choice}" for choice in choices]

        # 이미지와 텍스트 인코딩
        try:
            # 이미지 인코딩
            image_inputs = self.processor(
                images=image,
                return_tensors="pt",
                do_rescale=True,
                do_normalize=True
            )

            # 텍스트 인코딩
            text_inputs = self.processor(
                text=texts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=77
            )

            result = {
                'pixel_values': image_inputs.pixel_values.squeeze(0),
                'input_ids': text_inputs.input_ids,
                'attention_mask': text_inputs.attention_mask,
                'id': row["ID"]
            }

            if self.is_train:
                label_to_index = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
                result['labels'] = torch.tensor(label_to_index[row["answer"]], dtype=torch.long)

            return result

        except Exception as e:

            raise e

In [14]:
def collate_fn(batch):
    """배치 데이터를 적절히 결합"""
    if not batch:
        return None

    # 배치 내 모든 텐서의 크기를 맞춤
    pixel_values = torch.stack([item['pixel_values'] for item in batch])

    # 텍스트 입력들의 최대 길이 찾기
    max_length = 0
    for item in batch:
        max_length = max(max_length, item['input_ids'].shape[1])

    # 패딩을 적용하여 모든 텍스트 입력의 길이를 맞춤
    padded_input_ids = []
    padded_attention_mask = []

    for item in batch:
        input_ids = item['input_ids']  # shape: (4, seq_len)
        attention_mask = item['attention_mask']  # shape: (4, seq_len)

        current_length = input_ids.shape[1]
        if current_length < max_length:
            # 패딩 적용
            pad_size = max_length - current_length
            input_ids = torch.nn.functional.pad(input_ids, (0, pad_size), value=0)
            attention_mask = torch.nn.functional.pad(attention_mask, (0, pad_size), value=0)

        padded_input_ids.append(input_ids)
        padded_attention_mask.append(attention_mask)

    # 배치 차원으로 결합
    all_input_ids = torch.cat(padded_input_ids, dim=0)  # shape: (batch_size * 4, max_length)
    all_attention_mask = torch.cat(padded_attention_mask, dim=0)  # shape: (batch_size * 4, max_length)

    ids = [item['id'] for item in batch]

    result = {
        'pixel_values': pixel_values,
        'input_ids': all_input_ids,
        'attention_mask': all_attention_mask,
        'ids': ids
    }

    if 'labels' in batch[0]:
        labels = torch.stack([item['labels'] for item in batch])
        result['labels'] = labels

    return result

In [15]:
class CLIPVQAModel(nn.Module):
    """CLIP 기반 VQA 모델"""
    def __init__(self, clip_model_name, num_choices=4, lora_config=None):
        super().__init__()
        self.clip_model = CLIPModel.from_pretrained(
            clip_model_name,
            use_safetensors=False,
            #load_in_4bit=True,  # 또는 load_in_8bit=True
            #device_map="auto",
            )
        self.num_choices = num_choices

        #self.clip_model = prepare_model_for_kbit_training(self.clip_model)

            # LoRA 적용
        self.clip_model = get_peft_model(self.clip_model, lora_config)

        print("LoRA 모델 정보:")
        self.clip_model.print_trainable_parameters()

    def forward(self, pixel_values, input_ids, attention_mask):
        batch_size = pixel_values.shape[0]

        # 이미지 특성 추출
        image_features = self.clip_model.get_image_features(pixel_values)

        # 텍스트 특성 추출
        text_features = self.clip_model.get_text_features(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # 특성 정규화
        image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
        text_features = text_features / text_features.norm(p=2, dim=-1, keepdim=True)

        # 텍스트 특성을 (batch_size, num_choices, embedding_dim)로 재구성
        text_features = text_features.view(batch_size, self.num_choices, -1)

        # 유사도 계산
        # image_features: (batch_size, embedding_dim)
        # text_features: (batch_size, num_choices, embedding_dim)
        logits = torch.bmm(
            image_features.unsqueeze(1),
            text_features.transpose(1, 2)
        ).squeeze(1)

        # 온도 스케일링 적용
        logit_scale = self.clip_model.logit_scale.exp()
        logits = logit_scale * logits

        return logits

In [16]:
scaler = GradScaler('cuda')
def train_epoch(model, dataloader, optimizer, criterion, device):
    """한 에포크 훈련"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    progress_bar = tqdm(dataloader, desc="Training")

    for batch in progress_bar:
        if batch is None:
            continue

        # 데이터를 디바이스로 이동
        pixel_values = batch['pixel_values'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        # 순전파
        #logits = model(pixel_values, input_ids, attention_mask)
        #loss = criterion(logits, labels)

        # 역전파
        #loss.backward()
        #optimizer.step()

        with autocast('cuda'):
          logits=model(pixel_values, input_ids, attention_mask)
          loss = criterion(logits, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # 통계 업데이트
        total_loss += loss.item()
        pred = logits.argmax(dim=1)
        correct += (pred == labels).sum().item()
        total += labels.size(0)

        # 진행률 표시줄 업데이트
        progress_bar.set_postfix({
            'loss': total_loss / (progress_bar.n + 1),
            'acc': 100. * correct / total
        })

    avg_loss = total_loss / len(dataloader)
    accuracy = 100. * correct / total

    return avg_loss, accuracy

def validate_epoch(model, dataloader, criterion, device):
    """한 에포크 검증"""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        progress_bar = tqdm(dataloader, desc="Validation")

        for batch in progress_bar:
            if batch is None:
                continue

            pixel_values = batch['pixel_values'].to(device)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            #logits = model(pixel_values, input_ids, attention_mask)
            #loss = criterion(logits, labels)

            with autocast('cuda'):
              logits = model(pixel_values, input_ids, attention_mask)
              loss = criterion(logits, labels)

            total_loss += loss.item()
            pred = logits.argmax(dim=1)
            correct += (pred == labels).sum().item()
            total += labels.size(0)

            progress_bar.set_postfix({
                'loss': total_loss / (progress_bar.n + 1),
                'acc': 100. * correct / total
            })

    avg_loss = total_loss / len(dataloader)
    accuracy = 100. * correct / total

    return avg_loss, accuracy

In [17]:
def create_lora_config(config):
    """LoRA 설정 생성"""
    lora_config = LoraConfig(
        r=config.lora_r,
        lora_alpha=config.lora_alpha,
        target_modules=config.lora_target_modules,
        lora_dropout=config.lora_dropout,
        bias="none",
        task_type=TaskType.FEATURE_EXTRACTION,
    )
    return lora_config

In [ ]:
def main():
    # 데이터 로드
    train_df = pd.read_csv(config.train_csv_path)
    test_df = pd.read_csv(config.test_csv_path)

    # 훈련/검증 분할
    train_data, val_data = train_test_split(
        train_df,
        test_size=config.validation_split,
        random_state=seed,
        stratify=train_df['answer']
    )

    print(f"Training samples: {len(train_data)}")
    print(f"Validation samples: {len(val_data)}")
    print(f"Test samples: {len(test_df)}")

    # 프로세서 로드
    processor = CLIPProcessor.from_pretrained(config.processor_name)

    # 데이터셋 생성
    train_dataset = VQADataset(train_data, processor, config.image_base_path, is_train=True)
    val_dataset = VQADataset(val_data, processor, config.image_base_path, is_train=True)
    test_dataset = VQADataset(test_df, processor, config.image_base_path, is_train=False)

    # 데이터로더 생성
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=2
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=config.batch_size,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=2
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=config.batch_size,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=2
    )
    lora_config = create_lora_config(config)
    # 모델 생성
    model = CLIPVQAModel(config.model_name, config.num_choices,lora_config).to(config.device)

    # 옵티마이저 및 손실 함수
    optimizer = optim.AdamW(
        model.parameters(),
        lr=config.learning_rate,
        weight_decay=config.weight_decay
    )

    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    # 스케줄러 (선택사항)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=2,
        verbose=True
    )

    # 훈련 루프
    best_val_acc = 0
    patience_counter = 0

    print("=" * 60)
    print("Starting Training...")
    print("=" * 60)

    for epoch in range(config.num_epochs):
        print(f"\nEpoch {epoch + 1}/{config.num_epochs}")
        print("-" * 40)

        # 훈련
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, config.device)

        # 검증
        val_loss, val_acc = validate_epoch(model, val_loader, criterion, config.device)

        # 스케줄러 업데이트
        scheduler.step(val_loss)

        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        # 베스트 모델 저장
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0

            if config.save_best_model:
                torch.save(model.state_dict(), config.model_save_path)
                print(f"New best model saved with validation accuracy: {val_acc:.2f}%")
        else:
            patience_counter += 1

        # Early stopping
        if patience_counter >= config.patience:
            print(f"Early stopping triggered after {epoch + 1} epochs")
            break
    print('Train finish...')
    '''
    # 베스트 모델 로드
    if config.save_best_model and os.path.exists(config.model_save_path):
        model.load_state_dict(torch.load(config.model_save_path))
        print(f"Loaded best model with validation accuracy: {best_val_acc:.2f}%")

    # 테스트 추론
    print("\n" + "=" * 60)
    print("Starting Inference...")
    print("=" * 60)

    model.eval()
    predictions = []
    ids = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            if batch is None:
                continue

            pixel_values = batch['pixel_values'].to(config.device)
            input_ids = batch['input_ids'].to(config.device)
            attention_mask = batch['attention_mask'].to(config.device)

            logits = model(pixel_values, input_ids, attention_mask)
            pred_indices = logits.argmax(dim=1).cpu().numpy()

            label_map = {0: "A", 1: "B", 2: "C", 3: "D"}
            batch_predictions = [label_map[idx] for idx in pred_indices]

            predictions.extend(batch_predictions)
            ids.extend(batch['ids'])

    # 제출 파일 생성
    submission_df = pd.DataFrame({'ID': ids, 'answer': predictions})
    submission_df.to_csv(config.output_submission_path, index=False)

    print(f"✅ Submission saved to {config.output_submission_path}")
    print(f"📊 Final Results:")
    print(f"   - Best Validation Accuracy: {best_val_acc:.2f}%")
    print(f"   - Total Predictions: {len(predictions)}")
    '''
if __name__ == "__main__":
    main()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Training samples: 118938
Validation samples: 20990
Test samples: 852


preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

Training:   0%|          | 0/1859 [00:00<?, ?it/s]

Validation:   0%|          | 0/328 [00:00<?, ?it/s]

Train Loss: 0.4523, Train Acc: 97.84%
Val Loss: 0.8511, Val Acc: 76.16%
Early stopping triggered after 5 epochs
Train finish...
Loaded best model with validation accuracy: 78.37%

Starting Inference...


Testing:   0%|          | 0/14 [00:00<?, ?it/s]

✅ Submission saved to submission_pluto.csv
📊 Final Results:
   - Best Validation Accuracy: 78.37%
   - Total Predictions: 852


CLIP paper: https://arxiv.org/abs/2103.00020

VIsual7W paper: https://arxiv.org/abs/1511.03416

https://github.com/yukezhu/visual7w-toolkit

https://ai.stanford.edu/~yukez/visual7w/